1 задание:
Целевой переменной (target) является churn(что подразумевает потерю клиентов) типа данных bool. Если переменная равна «False» - данный абонент всё ещё является клиентом компании, «True» - не является. 
Основная задача прогнозной модели - определить лояльных абонентов.

In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Подключите необходимые библиотеки.

In [ ]:
 import numpy as np
 import pandas as pd
 import matplotlib.pyplot as plt
 import seaborn as sns
 sns.set(rc={'figure.figsize':(10, 8)}); # you can change this if needed

Загрузите данные. Они находятся в директории input.

In [ ]:
 df = pd.read_csv('../input/bigml_59c28831336c6604c800002a.csv')
 df

Каково распределение абонентов по целевому признаку **churn**? Назовите процент лояльных абонентов.

In [ ]:
df['churn'].value_counts(normalize=True).plot(kind = 'bar')
df['churn'].value_counts(normalize=True)

 85,5% лояльных абонентов 

Жители какого штата говорят по телефону больше всего? меньше всего? Обратите внимание, что нас интересуют все виды звонков.

In [ ]:
df['all calls']=df['total day calls']+df['total eve calls']+df['total night calls']+df['total intl calls']+df['customer service calls']
print("state with max calls:",df.groupby('state')['all calls'].sum().idxmax(),df.groupby('state')['all calls'].sum().max())
print("state with min calls:",df.groupby('state')['all calls'].sum().idxmin(),df.groupby('state')['all calls'].sum().min())

Жители штата WV говорят больше всего по телефону, а штата CA - меньше всего

Верно ли, что люди, совершающие много международных звонков, приобретают **international plan**? Не забывайте о визуализации информации.

In [ ]:
df.groupby('total intl calls')['international plan'].value_counts(normalize="True").drop(labels='no', level=1).plot(kind = 'bar')


Верно, поскольку с увеличением количества звонков, процент людей с международным планом скорее увеличивается, чем уменьшается

Как связаны наличие голосовой почты и международного плана с лояльностью абонентов? Проверьте статистическую значимость этих связей.

In [ ]:
from scipy.stats import chi2_contingency, fisher_exact
chi2_contingency(pd.crosstab(df['voice mail plan'], df['churn']))
#chi2_contingency(pd.crosstab(df['international plan'], df['churn']))

In [ ]:
chi2_contingency(pd.crosstab(df['international plan'], df['churn']))

Из достаточно малого значения P-value следует статистическая зависимость голосовой почты и международного плана с лояльностью абонентов

Что можно сказать о взаимосвязи количества обращений в колл-центр и лояльности? Не забывайте о визуализации информации.

In [ ]:
from scipy.stats import pearsonr, spearmanr, kendalltau
r = pearsonr(df['customer service calls'], df['churn'])
print('Pearson correlation:', r[0], 'p-value:', r[1])
sns.heatmap(pd.crosstab(df['customer service calls'], df['churn']), 
            cmap="YlGnBu", annot=True, cbar=False);

Они малосвязаны. Коэффициент корреляции невелик, p-значение велико

Какие из числовых признаков имеют наибольшую степень корреляции? Почему?

In [ ]:
numeric = ['account length','area code','number vmail messages','total day minutes','total day calls','total day charge','total eve minutes','total eve calls','total eve charge','total night minutes','total night calls','total night charge','total intl minutes','total intl calls','total intl charge','customer service calls']
sns.pairplot(df[numeric]);
df[numeric].corr(method='spearman')

Наибольшую степень корреляции имеют признаки количества звонков за какое-либо время и расходы на звонки за какое-либо время. Скорее всего плата привязана не столько к длительности звонка, сколько к его наличию и отсутствию

Связано ли время пользования аккаунтом с количеством обращений в колл-центр?

In [ ]:
from scipy.stats import pearsonr, spearmanr, kendalltau
r = pearsonr(df['customer service calls'], df['account length'])
print('Pearson correlation:', r[0], 'p-value:', r[1])

Да, поскольку коэффициент достаточно велик, а p-значение достаточно мало

Какие ещё инсайты можно найти в имеющихся данных? Творческое задание.

In [ ]:
new_values = {False:0,True:1}
df['churn'] = df['churn'].map(new_values)
numerical = ['account length','area code','number vmail messages','total day minutes','total day calls','total day charge','total eve minutes','total eve calls','total eve charge','total night minutes','total night calls','total night charge','total intl minutes','total intl calls','total intl charge','customer service calls','churn']
sns.heatmap(df[numerical].corr(method='spearman'));


Переведем тип таргет значения из булевого в численный. Построив таблицу функцией heatmap заметим, что помимо количества звонков в службу поддержки статистическое влияние оказывает плата за день и время разговоров за день и вечер